# NOTES
---
## Present State
* Code is presently oriented to calculate unit commitment for the base case (D = wind + slack + sum(generators)) and all scenarios for the full optimization horizon of 1 week and export results if desired.
    * Each day a new optimization problem is solved (generation amounts & unit commitment results for the last hour are passed to the next day via g0 and u0 respectively).
    * Set decision flags to 1 and alter scenario constants (e.g. Coal Curtailment factor) to examine and solve various cases.
    
* Results Include:
  * Wind curtailment, system cost, and C02 Emitted
    * **Average Marginal Price & Profit (both for generators and wind) are presently dubious**  
  * I've added functionality to export the majority of the tracked results into Excel files. Just set the variable "save" to 1 and specify the file path you want for your results (default is "Results/Scenario_X").

---  
 ## Not Incorporated  
* Start-up & shut-down horizons (constraints  6-9; worth 5% of
total grade)
---   

# Setup and Data Import

A reference importing data from csv file to numpy arrays. You can use your own data processing script.

In [ ]:
import numpy as np
import pandas as pd

# Data import and pre-processing
generator_data_raw = pd.read_csv('generator_data.csv')
gen_data = np.array(generator_data_raw)
demand_data_raw = pd.read_csv('wind_and_demand.csv')
demand_data = np.array(demand_data_raw)
D_raw = demand_data[0:,3]
DEMAND = np.reshape(D_raw, (7,24))
wind_CF_raw = pd.read_csv('wind_and_demand.csv')
wind_CF = np.array(wind_CF_raw)
w_CF_raw = wind_CF[0:,2]
w_CF = np.reshape(w_CF_raw, (7,24))

# Extract individual generator variables
g_max = gen_data[0:,2]
g_max = g_max.astype('float')
g_min = gen_data[0:,3]
g_min = g_min.astype('float')

# Fuel Types
f_type = gen_data[0:,4]

# Generator Operation Constraints
Tup = gen_data[0:,5]     ## Min Up Time
Tup = Tup.astype('float')
Tdown = gen_data[0:,6]   ## Min Down Time
Tdown = Tdown.astype('float')
rrate = gen_data[0:,7]
rrate = rrate.astype('float')

# Generator Cost Parameters
SUC = gen_data[0:,8]     ##Start-up cost
SUC = SUC.astype('float')
NLC = gen_data[0:,9]     ##No Load cost
NLC = NLC.astype('float')

a = gen_data[0:,10] ## linear Dispatch Cost Coefficient
a = a.astype('float')
b = gen_data[0:,11] ## Quadratic Dispatch Cost Coefficient
b = b.astype('float')

la = gen_data[0:,12] ## lambda for carbon tax
la = la.astype('float')

# Variable Declaration

In [ ]:
import cvxpy as cp
import gurobipy

# Define parameters
N = len(gen_data); # Number of generators (76)
D = 7;  # Number of Days (7)
H = 24; # Number of hours (24)

g = cp.Variable((N,H), nonneg = True)    # Generator Dispatch Qunatity
s = cp.Variable((H), nonneg = True)      # Slack Variable (global across whole system)
w = cp.Variable((H), nonneg = True)      # Wind generation Quantity
R = cp.Variable((N,H), nonneg = True)    # Reserve Quantity

#Binary Decision Variables
u = cp.Variable((N,H), boolean = True)   # Commitment Status
v = cp.Variable((N,H), boolean = True)   # Start-Up Status
z = cp.Variable((N,H), boolean = True)   # Shut-Down Status

# State Initialization Variables (Updated each day)
u0 = np.zeros((N))  ## Initial Commitment Status (All Generators are off to start of first day)
g0 = np.zeros((N)) ## Value of generation for each generator in the last hour of each day

########################### Scenario Constants  ######################
CoC = 45 # Cost of Carbon
w_cred = 26 ## Wind Production Tax credit $26/MWh

########################### Result Variables ######################
wind_curtailment = np.zeros((H)) # equals (W - w[t]) / W
Total_Cost = 0.0 # Running Cost

# Optimization Code & Results

In [ ]:
import os

############################ Table Headers & Printing Variables #############################
# Define row names and headers
row_Names = []
col_Names = []
DAILY_HEADERS = ['Daily CO2 Emissions','Daily Operating Cost']

for i in range(N):
    row_Names.append('Unit_{}'.format(i+1))

for t in range(H):
    col_Names.append('Hour_{}'.format(t+1))

# Export control variable and directory creation
save = 1
path = '_Results/Normal/Scenario_X'
if not os.path.exists(path):
    os.makedirs(path)
##############################################################################################

########################## Scenario Modifiers ##########################
W = 2000 # Maximum MW capacity for Wind (Scenario 1)

### Scenario Flags ###
COAL_RETIREMENT = 0   # Set to 1 to apply designated coal retirement
CARBON_TAX = 0         # Set to 1 to apply carbon tax
TAX_INCENTIVE = 0      # Set to 1 to apply wind tax incentive

#Coal Retirement Scenario
curtail_factor = 0.5
if(COAL_RETIREMENT == 1):
    for i in range(N):
        if(f_type[i] == ' BIT ' or f_type[i] == 'BIT' or f_type[i] == ' SUB ' or f_type[i] == 'SUB'):
            g_min[i] = 0.0
            g_max[i] = curtail_factor * g_max[i]
########################################################################

######################## CVXPY Dictionary Setup ##########################
con_set =  {}
obj_set =  {}
prob_set = {}

for d in range(D):
    con_set[d] =  []
    obj_set[d] =  []
    prob_set[d] = []
#########################################################################

#######################################################################################################################################################
##################################################################### CODE BODY ######################################################################

# Go through each day of the week
for d in range(D):
    Daily_CO2_Amnt = 0.0 # Daily C02 Emmissions
    if(CARBON_TAX == 1 and TAX_INCENTIVE == 1):
        obj_set[d].append(cp.Minimize(cp.sum(a@g + (b @ (g**2)) + NLC @ u + SUC @ v + 9000*s - w_cred*w) + CoC*cp.sum(la@g))) ##### + CoC*cp.sum(la@g))) NEED CYRIL's CODE
    if(CARBON_TAX == 1 and TAX_INCENTIVE == 0):
        obj_set[d].append(cp.Minimize(cp.sum(a@g + (b @ (g**2)) + NLC @ u + SUC @ v + 9000*s) + CoC*cp.sum(la@g)))
    if(CARBON_TAX == 0 and TAX_INCENTIVE == 1):
        obj_set[d].append(cp.Minimize(cp.sum(a@g + (b @ (g**2)) + NLC @ u + SUC @ v + 9000*s - w_cred*w))) ### NEED CYRIL'S CODE
    else:
        obj_set[d].append(cp.Minimize(cp.sum(a@g + (b @ (g**2)) + NLC @ u + SUC @ v + 9000*s)))

    con_set[d].append(DEMAND[d] == sum(g) + s + w)

    # Go through each hour of the day
    for t in range(H):
        con_set[d].append(w[t] <= w_CF[d][t] * W)             # Wind capacity requirement

        # Go through each generator
        for i in range(N):
            con_set[d].append(g[i][t] <= g_max[i] * u[i][t])  # maximum generation limits
            con_set[d].append(g[i][t] >= g_min[i] * u[i][t])  # minimum generation limits

            #Constraints 12 & 13
            #R[i][t] <= (g_max[i]*u[i][t]) - g[i][t]           # Reserve - Gmax Constraint (not Necessary?)
            #R[i][t] <= rrate[i]                               # Reserve - RR Constraint (not Necessary?)

            if(d == 0):
                if(t == 0):
                    # for the first period, check with initial commitment status on the first day
                    con_set[d].append(v[i][t] - z[i][t] == u[i][t] - u0[i])
                    con_set[d].append(v[i][t] + z[i][t] <= 1)
                else:
                    # for other periods, check difference between two commitment status
                    con_set[d].append(v[i][t] - z[i][t] == u[i][t] - u[i][t-1]) ## Gen Start-up Criteria
                    con_set[d].append(v[i][t] + z[i][t] <= 1)                   ## Gen Shut-down Criteria

                    con_set[d].append(g[i][t] - g[i][t-1] <= rrate[i] + g_min[i] * v[i][t] ) ## ramp up Criteria
                    con_set[d].append(g[i][t] - g[i][t-1] >= -g_min[i] * z[i][t] - rrate[i]) ## ramp down

            else: #pass info from the previous day to the next
                if(t == 0):
                     # for the first period, check with initial commitment status on the first day
                    con_set[d].append(v[i][t] - z[i][t] == u[i][t] - u0[i])
                    con_set[d].append(v[i][t] + z[i][t] <= 1)

                    con_set[d].append(g[i][t] - g0[i] <= rrate[i] + g_min[i] * v[i][t] ) ## ramp up Criteria
                    con_set[d].append(g[i][t] - g0[i] >= -g_min[i] * z[i][t] - rrate[i]) ## ramp down

                else:
                     # for other periods, check difference between two commitment status
                    con_set[d].append(v[i][t] - z[i][t] == u[i][t] - u[i][t-1]) ## Gen Start-up Criteria
                    con_set[d].append(v[i][t] + z[i][t] <= 1)                   ## Gen Shut-down Criteria

                    con_set[d].append(g[i][t] - g[i][t-1] <= rrate[i] + g_min[i] * v[i][t] ) ## ramp up Criteria
                    con_set[d].append(g[i][t] - g[i][t-1] >= -g_min[i] * z[i][t] - rrate[i]) ## ramp down

        #Constraint 11 (Reserve 3 + 5 rule)
        con_set[d].append(sum(R[:,t]) >= (0.03 * DEMAND[d][t]) + (0.05 * w[t]))

    # Solve the optimization problm for each day
    prob_set[d].append(cp.Problem(obj_set[d][0], con_set[d]))
    prob_set[d][0].solve(solver = "GUROBI", verbose=True)
    prob_set[d][0].solve()


  #################### Determine Additional Results ####################
    for t in range(H):
        wind_curtailment[t] = ((((w_CF[d][t]*W) - w[t].value)) / (w_CF[d][t]*W))

    Daily_CO2_Amnt = sum(la@g.value)

    ########################### UPDATE FOR NEXT DAY ############################
    # Update State Initialization variables
    u0 = u
    g0 = g
    # Update Total Running Cost
    Total_Cost += obj_set[d][0].value


#######################################################################################################################################################
#######################################################################################################################################################

###################################################################### PRINT & SAVE RESULTS ##################################################################
    print("\n####################################### DAY %i Data #######################################" % (d+1))
    # print out results
    with pd.option_context('display.max_rows', None,
                           'display.max_columns', None,
                           'display.precision', 4,
                           ):

         ########### System  Results ###########

        print("\n Total Daily CO2 Emissionst: $%.2f" % Daily_CO2_Amnt)
        print("\n Total Daily Operating Cost: $%.2f" % obj_set[d][0].value)
        DAILY_RESULTS = pd.DataFrame([Daily_CO2_Amnt, obj_set[d][0].value],DAILY_HEADERS)

        print("\n Determined Slack results: ")
        ST = pd.DataFrame(s.value, col_Names)
        print(ST.round(2))

        ########### Wind Gen Related Results ###########
        print("\n Determined Wind Generation results: ")
        WT = pd.DataFrame(w.value, col_Names)
        print(WT.round(2))

        print("\n Wind Curtailment Ratio results: ")
        WCT = pd.DataFrame(wind_curtailment, col_Names)
        print(WCT.round(2))

        ########### Generator Related Results ###########
        print("\n Generator dispatch results: ")
        GT = pd.DataFrame(g.value,  row_Names, col_Names)
        print(GT.round(2))

        print("\n Reserve Quantity: ")
        RT = pd.DataFrame(R.value, row_Names, col_Names)
        print(RT.round(2))

        print("\n Generator commitment results: ")
        UT = pd.DataFrame(u.value,  row_Names, col_Names)
        print(UT.round(1))

        print("\n Generator start-up results: ")
        VT = pd.DataFrame(v.value,  row_Names, col_Names)
        print(VT.round(1))

        print("\n Generator shut-down results: ")
        ZT = pd.DataFrame(z.value,  row_Names, col_Names)
        print(ZT.round(1))

    if(save == 1):
        with pd.ExcelWriter(path+"/Day%i_Data.xlsx"%(d+1), engine="openpyxl") as writer:
            DAILY_RESULTS.to_excel(writer, sheet_name="Daily Results", index=True)
            ST.to_excel(writer, sheet_name="Slack", index=True)
            WT.to_excel(writer, sheet_name="Wind Generation", index=True)
            WCT.to_excel(writer, sheet_name="Wind Reserve Ratio", index=True)
            GT.to_excel(writer, sheet_name="Generator Dispatch", index=True)
            RT.to_excel(writer, sheet_name="Reserve Quantity", index=True)
            UT.to_excel(writer, sheet_name="Unit Commitment", index=True)
            VT.to_excel(writer, sheet_name="Generator Start-Up", index=True)
            ZT.to_excel(writer, sheet_name="Generator Shut-Down", index=True)


print("\n\nTotal Cost Across Entire Optimization Horizon: $%.2f" % Total_Cost)
######################################################################################################################################################

########################################################### IN PROGRESS: CONSTRAINTS 6-9  ############################################################
        #not sure if these are correct
        #if(u[i][t] == 1 and v[i][t] == 1):
         #   SU[i] = max(0, Tup[i] - t + 1)
         #   SD[i] = 0

        #if(u[i][t] == 0 and z[i][t] == 1):
         #   SD[i] =  max(0, Tdown[i] - t + 1)
          #  SU[i] = 0


       #I think these are correct
       # tau_1 = max(t - Tup[i] + 1, 1);
       # tau_2 = max(t - Tdown[i] + 1, 1);
       # con_set_1.append(sum(v[i][tau_1:t]) <= u[i][t])     # Min Up-time Criteria
       # con_set_1.append(sum(z[i][tau_2:t]) <= 1 - u[i][t]) # Min Down-time Criteria
######################################################################################################################################################

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Dec 16 03:26:10 PM: Your problem has 9168 variables, 10841 constraints, and 0 parameters.
(CVXPY) Dec 16 03:26:11 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 16 03:26:11 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 16 03:26:11 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 16 03:26:11 PM: Compiling problem (target solver=GUROBI).
(CVXPY) Dec 16 03:26:11 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrix

# Dual Value Section (for determining Marginal Price & Profits)

In [ ]:
#####################################################  DUAL VARIABLE ###########################################################
Total_Cost = 0.0 # Reset Running Cost
############################ Table Headers & Printing Variables #############################
# Define row names and headers
row_Names = []
col_Names = []
DAILY_HEADERS = ['Daily CO2 Emissions','Daily Operating Cost','Total Generator Revenue', 'Total Generator Profit','Total Wind Profit']

for i in range(N):
    row_Names.append('Unit_{}'.format(i+1))

for t in range(H):
    col_Names.append('Hour_{}'.format(t+1))

# Export control variable and directory creation
save = 1
path = '_Results/Dual/Scenario_X'
if not os.path.exists(path):
    os.makedirs(path)
##############################################################################################

########################## Scenario Modifiers ##########################
W = 2000 # Maximum MW capacity for Wind (Scenario 1)

### Scenario Flags ###
COAL_RETIREMENT = 0   # Set to 1 to apply designated coal retirement
CARBON_TAX = 0         # Set to 1 to apply carbon tax
TAX_INCENTIVE = 0      # Set to 1 to apply wind tax incentive

#Coal Retirement Scenario
curtail_factor = 0.5
if(COAL_RETIREMENT == 1):
    for i in range(N):
        if(f_type[i] == ' BIT ' or f_type[i] == 'BIT' or f_type[i] == ' SUB ' or f_type[i] == 'SUB'):
            g_min[i] = 0.0
            g_max[i] = curtail_factor * g_max[i]
########################################################################

######################## CVXPY Dictionary Setup ##########################
con_set_2 =  {}
obj_set_2 =  {}
prob_set_2 = {}

for d in range(D):
    con_set_2[d] =  []
    obj_set_2[d] =  []
    prob_set_2[d] = []
#########################################################################

#######################################################################################################################################################
##################################################################### CODE BODY ######################################################################

# Go through each day of the week
for d in range(D):
    Avg_Price = 0.0 # Daily Average Electricity Price (acquired from dual variable)
    Daily_CO2_Amnt = 0.0 # Daily C02 Emmissions
    if(CARBON_TAX == 1 and TAX_INCENTIVE == 1):
        obj_set_2[d].append(cp.Minimize(cp.sum(a@g + (b @ (g**2)) + NLC @ u.value + SUC @ v.value + 9000*s))) ##### + CoC*cp.sum(la@g))) NEED CYRIL's CODE
    if(CARBON_TAX == 1 and TAX_INCENTIVE == 0):
        obj_set_2[d].append(cp.Minimize(cp.sum(a@g + (b @ (g**2)) + NLC @ u.value + SUC @ v.value + 9000*s) + CoC*cp.sum(la@g)))
    if(CARBON_TAX == 0 and TAX_INCENTIVE == 1):
        obj_set_2[d].append(cp.Minimize(cp.sum(a@g + (b @ (g**2)) + NLC @ u.value + SUC @ v.value + 9000*s - 26*w))) ### NEED CYRIL'S CODE
    else:
        obj_set_2[d].append(cp.Minimize(cp.sum(a@g + (b @ (g**2)) + NLC @ u.value + SUC @ v.value + 9000*s)))
    con_set_2[d].append(DEMAND[d] == sum(g) + s + w)
    # Go through each hour of the day
    for t in range(H):
        con_set_2[d].append(w[t] <= w_CF[d][t] * W)             # Wind capacity requirement
        # Go through each generator
        for i in range(N):
            con_set_2[d].append(g[i][t] <= g_max[i] * u.value[i][t])  # maximum generation limits
            con_set_2[d].append(g[i][t] >= g_min[i] * u.value[i][t])  # minimum generation limits
            #Constraints 12 & 13
            #R[i][t] <= (g_max[i]*u[i][t]) - g[i][t]           # Reserve - Gmax Constraint (not Necessary?)
            #R[i][t] <= rrate[i]                               # Reserve - RR Constraint (not Necessary?)

            #if(d == 0):
                #if(t == 0):
                    # for the first period, check with initial commitment status on the first day
                    #con_set[d].append(v[i][t] - z[i][t] == u[i][t] - u0[i])
                    #con_set[d].append(v[i][t] + z[i][t] <= 1)
                #else:
                    # for other periods, check difference between two commitment status
                #con_set_2[d].append(v[i][t].value - z[i][t].value == u[i][t].value - u[i][t-1].value) ## Gen Start-up Criteria
                #con_set_2[d].append(v[i][t].value + z[i][t].value <= 1)                   ## Gen Shut-down Criteria

                #con_set_2[d].append(g[i][t] - g[i][t-1] <= rrate[i] + g_min[i] * v.value[i][t] ) ## ramp up Criteria
                #con_set_2[d].append(g[i][t] - g[i][t-1] >= -g_min[i] * z.value[i][t] - rrate[i]) ## ramp down

            #else: #pass info from the previous day to the next
             #   if(t == 0):
                     # for the first period, check with initial commitment status on the first day
                    #con_set_2[d].append(v.value[i][t] - z.value[i][t] == u.value[i][t] - u0.value[i])
                    #con_set_2[d].append(v.value[i][t] + z.value[i][t] <= 1)

                    #con_set_2[d].append(g[i][t] - g0[i] <= rrate[i] + g_min[i] * v.value[i][t]) ## ramp up Criteria
                    #con_set_2[d].append(g[i][t] - g0[i] >= -g_min[i] * z.value[i][t] - rrate[i]) ## ramp down

              #  else:
                     # for other periods, check difference between two commitment status
                    #con_set_2[d].append(v.value[i][t] - z.value[i][t] == u.value[i][t] - u.value[i][t-1]) ## Gen Start-up Criteria
                    #con_set_2[d].append(v.value[i][t] + z.value[i][t] <= 1)                   ## Gen Shut-down Criteria

                    #con_set_2[d].append(g[i][t] - g[i][t-1] <= rrate[i] + g_min[i] * v.value[i][t] ) ## ramp up Criteria
                    #con_set_2[d].append(g[i][t] - g[i][t-1] >= -g_min[i] * z.value[i][t] - rrate[i]) ## ramp down

        #Constraint 11 (Reserve 3 + 5 rule)
        #con_set_2[d].append(sum(R[:,t]) >= (0.03 * DEMAND[d][t]) + (0.05 * w[t]))
    # Solve the optimization problm for each day
    prob_set_2[d].append(cp.Problem(obj_set_2[d][0], con_set_2[d]))
    prob_set_2[d][0].solve(solver = "GUROBI", verbose=True)
    prob_set_2[d][0].solve()

  ############################ Determine Additional Results #############################
    for t in range(H):
        wind_curtailment[t] = ((((w_CF[d][t]*W) - w.value[t])) / (w_CF[d][t]*W))
        Avg_Price += (-con_set_2[d][0].dual_value[t])

    Avg_Price = (1/24) * Avg_Price
    Daily_Revenue_Gen = Avg_Price*(sum(sum(g.value)))
    Daily_Profit_Gen = Daily_Revenue_Gen - obj_set_2[d][0].value
    Daily_Profit_Wind = Avg_Price * sum(w.value)
    Daily_CO2_Amnt = sum(la@g.value)

    ########################### UPDATE FOR NEXT DAY ############################
    # Update State Initialization variables
    u0 = u.value
    g0 = g
    # Update Total Running Cost
    Total_Cost += obj_set_2[d][0].value


#######################################################################################################################################################
#######################################################################################################################################################

###################################################################### PRINT & SAVE RESULTS ##################################################################
    print("\n####################################### DAY %i Data #######################################" % (d+1))
    # print out results
    with pd.option_context('display.max_rows', None,
                           'display.max_columns', None,
                           'display.precision', 4,
                           ):

         ########### System  Results ###########
        print("\n Total Daily CO2 Emissions: $%.2f" % Daily_CO2_Amnt)
        print("\n Total Daily Operating Cost: $%.2f" % obj_set_2[d][0].value)
        print("\n Total Revenue for All Generators: $%.2f" % Daily_Revenue_Gen)
        print("\n Total Profit for All Generators: $%.2f" % Daily_Profit_Gen)
        print("\n Total Profit for Wind: $%.2f" % Daily_Profit_Wind)
        DAILY_RESULTS = pd.DataFrame([Daily_CO2_Amnt, obj_set_2[d][0].value,Daily_Revenue_Gen, Daily_Profit_Gen, Daily_Profit_Wind],DAILY_HEADERS)

        print("\n Hourly Marginal Price results: ")
        MP = pd.DataFrame(-con_set_2[d][0].dual_value, col_Names)
        print(MP)

        print("\n Determined Slack results: ")
        ST = pd.DataFrame(s.value, col_Names)
        print(ST.round(2))

        ########### Wind Gen Related Results ###########
        print("\n Determined Wind Generation results: ")
        WT = pd.DataFrame(w.value, col_Names)
        print(WT.round(2))

        print("\n Wind Curtailment Ratio results: ")
        WCT = pd.DataFrame(wind_curtailment, col_Names)
        print(WCT.round(2))

        ########### Generator Related Results ###########
        print("\n Generator dispatch results: ")
        GT = pd.DataFrame(g.value,  row_Names, col_Names)
        print(GT.round(2))

        print("\n Reserve Quantity: ")
        RT = pd.DataFrame(R.value, row_Names, col_Names)
        print(RT.round(2))

        print("\n Generator commitment results: ")
        UT = pd.DataFrame(u.value,  row_Names, col_Names)
        print(UT.round(1))

        print("\n Generator start-up results: ")
        VT = pd.DataFrame(v.value,  row_Names, col_Names)
        print(VT.round(1))

        print("\n Generator shut-down results: ")
        ZT = pd.DataFrame(z.value,  row_Names, col_Names)
        print(ZT.round(1))

    if(save == 1):
        with pd.ExcelWriter(path+"/Day%i_Data.xlsx"%(d+1), engine="openpyxl") as writer:
            DAILY_RESULTS.to_excel(writer, sheet_name="Daily Results", index=True)
            MP.to_excel(writer, sheet_name="Marginal Price", index=True)
            ST.to_excel(writer, sheet_name="Slack", index=True)
            WT.to_excel(writer, sheet_name="Wind Generation", index=True)
            WCT.to_excel(writer, sheet_name="Wind Reserve Ratio", index=True)
            GT.to_excel(writer, sheet_name="Generator Dispatch", index=True)
            RT.to_excel(writer, sheet_name="Reserve Quantity", index=True)
            UT.to_excel(writer, sheet_name="Unit Commitment", index=True)
            VT.to_excel(writer, sheet_name="Generator Start-Up", index=True)
            ZT.to_excel(writer, sheet_name="Generator Shut-Down", index=True)


print("\n\nTotal Cost Across Entire Optimization Horizon: $%.2f" % Total_Cost)
